In [5]:
import pickle
import mlflow
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("random-forest-hyperopt")

def load_pickle(filename):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)
    
from sklearn.pipeline import make_pipeline

In [6]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='/home/hilbert/zoomcamps/4_deployment/web-service-mlflow/mlruns/2', creation_time=1687549443027, experiment_id='2', last_update_time=1687549443027, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df["duration"] = df.duration.dt.total_seconds()
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

def prepare_dictionaries(df):
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    dicts = df[categorical + numerical].to_dict(orient="records")

    return dicts
    
dftr = read_dataframe("../../data/green_tripdata_2022-01.parquet")
dfvl = read_dataframe("../../data/green_tripdata_2022-02.parquet")

target = "duration"
ytr = dftr[target].values
yvl = dfvl[target].values

dict_train = prepare_dictionaries(dftr)
dict_val = prepare_dictionaries(dfvl)

In [8]:
with mlflow.start_run():

    params = dict(
        max_depth=20,
        n_estimators=100,
        min_samples_leaf=10,
        random_state=0
    )

    mlflow.log_params(params)
   
    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, ytr)
    ypr = pipeline.predict(dict_val)

    rmse = mean_squared_error(ypr, yvl, squared=False)
    print(params, rmse)
    mlflow.log_metric("rmse", rmse)

    mlflow.sklearn.log_model(pipeline, "model")



{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 15.100101175469167
